# Customer churn prediction with Network and Customer data using Amazon SageMaker XGBoost
_**Supervised Learning with Gradient Boosted Trees: A Binary Prediction Problem With Unbalanced Classes**_

---


## Background
Customer churn is a business challenge that all Communication Service Providers encounter on a daily basis. The goal of this machine learning notebook is to look at historical data from the network, subscribed products, usage, and pricing to predict the conditions under which a particular customer is likely to churn.The steps include:

* Data Transformation
* Training 
* Deploying 
* Model Evaluation

---

## Setup

_This notebook was created and tested on an ml.m4.xlarge notebook instance._

Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data.

### Install AWS SDK for Pandas

In [ ]:
# cell 01
!pip install awswrangler

## Import Libraries

In [ ]:
# cell 02
# Import libraries
import awswrangler as wr
import numpy as np                                # For matrix operations and numerical processing
import pandas as pd                               # For munging tabular data
import matplotlib.pyplot as plt                   # For charts and visualizations
from IPython.display import Image                 # For displaying images in the notebook
from IPython.display import display               # For displaying outputs in the notebook
from time import gmtime, strftime                 # For labeling SageMaker models, endpoints, etc.
import sys                                        # For writing outputs to notebook
import math                                       # For ceiling function
import json                                       # For parsing hosting outputs
import os                                         # For manipulating filepath names
import sagemaker                                  # Amazon SageMaker's Python SDK provides many helper functions
import zipfile                                    
import getpass
import time

## Set Bucket Name & Path

In [ ]:
# cell 03
bucket='tlc301assets' #It should be of the form tlc301-account_number
path = f"s3://tlc301assets/"

data = 'joined'
prefix = 'prepared'

# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

## Read Input Data and Start Data Preparation

The most common way of preventing this is to build models with the concept that a model shouldn't only be judged on its fit to the data it was trained on, but also on "new" data.  There are several different ways of operationalizing this, holdout validation, cross-validation, leave-one-out validation, etc.  For our purposes, we'll simply randomly split the data into 3 uneven groups.  The model will be trained on 70% of data, it will then be evaluated on 20% of data to give us an estimate of the accuracy we hope to have on "new" data, and 10% will be held back as a final testing dataset which will be used later on.

In [ ]:
# cell 04
df = wr.s3.read_parquet(path+data, dataset=True)

# Make Target Churn column binary
#change True. to 1 and False. to 0
df['churn'] = df['churn'].replace(regex=r'True.' , value='1')
df['churn'] = df['churn'].replace(regex=r'False.' , value='0')
df['phone'] = df['phone'].replace(regex=r'-' , value='')

# Drop another location column that is redundant 
df = df.drop('location5gcell', axis=1)

# One hot encoding
df = pd.get_dummies(df, columns=['location', 'intl_plan', 'vmail_plan'])

# Drop another location column that is redundant 
df = df.drop('partition', axis=1)

# Copy the original dataframe into a new dataframe
model_data = df.copy() 

# Randomly sort the data then split out first 70%, second 20%, and last 10%
train_data, validation_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data)), int(0.9 * len(model_data))])  

## Create training dataset for XGBoost algorithm
Amazon SageMaker's XGBoost container expects data in the libSVM or CSV data format.  For this example, we'll stick to CSV.  Note that the first column must be the target variable and the CSV should not include headers.  Also, notice that although repetitive it's easiest to do this after the train|validation|test split rather than before.  This avoids any misalignment issues due to random reordering.

In [ ]:
# Cell 05
# Export data to your local EBS volume.
pd.concat([train_data['churn'], train_data.drop(['churn'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)
pd.concat([validation_data['churn'], validation_data.drop(['churn'], axis=1)], axis=1).to_csv('validation.csv', index=False, header=False)
pd.concat([test_data['churn'], test_data.drop(['churn'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)

## Wait for THREE files to be generated from the previous step in the local directory (next to the notebook)

In [ ]:
# cell 06
# Now we'll copy the file to S3 for Amazon SageMaker's managed training to pickup.
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('validation.csv')

---

## Training (takes ~4 min)

There are several intricacies to understanding the algorithm, but at a high level, gradient boosted trees works by combining predictions from many simple models, each of which tries to address the weaknesses of the previous models.  By doing this, the collection of simple models can actually outperform large, complex models.

First we'll need to specify the ECR container location for Amazon SageMaker's implementation of XGBoost. Then, because we're training with the CSV file format, we'll create `s3_input`s that our training function can use as a pointer to the files in S3, which also specify that the content type is CSV.

First we'll need to specify training parameters to the estimator.  This includes:
1. The `xgboost` algorithm container
1. The IAM role to use
1. Training instance type and count
1. S3 location for output data
1. Algorithm hyperparameters

And then a `.fit()` function which specifies:
1. S3 location for output data.  In this case we have both a training and validation set which are passed in.

In [ ]:
# cell 07
container = sagemaker.image_uris.retrieve(region=boto3.Session().region_name, framework='xgboost', version='latest')

s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train/'.format(bucket, prefix), content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/validation/'.format(bucket, prefix), content_type='csv')

sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    instance_count=1, 
                                    instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/output'.format(bucket),
                                    sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        num_round=100)

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation}) 

---

## Deploy Model to Endpoint
Now that we've trained the `xgboost` algorithm on our data, let's deploy a model that's hosted behind a real-time endpoint.

In [ ]:
# cell 08
xgb_predictor = xgb.deploy(initial_instance_count=1,
                           instance_type='ml.m4.xlarge')

---
## Model Evaluation
There are many ways to compare the performance of a machine learning model, but let's start by simply comparing actual to predicted values.  In this case, we're simply predicting whether the customer churned (`1`) or not (`0`), which produces a simple confusion matrix.

First we'll need to determine how we pass data into and receive data from our endpoint.  Our data is currently stored as NumPy arrays in memory of our notebook instance.  To send it in an HTTP POST request, we'll serialize it as a CSV string and then decode the resulting CSV.

*Note: For inference with CSV format, SageMaker XGBoost requires that the data does NOT include the target variable.*

In [ ]:
# cell 09
xgb_predictor.serializer = sagemaker.serializers.CSVSerializer()


def predict(data, predictor, rows=500 ):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

predictions = predict(test_data.drop(['churn'], axis=1).to_numpy(), xgb_predictor)

pd.crosstab(index=test_data['churn'], columns=np.round(predictions), rownames=['actuals'], colnames=['predictions'])

Now we'll check our confusion matrix to see how well we predicted versus actuals.

So, of the ~500 potential customers, we predicted 247 would churn and 241 of them actually did.  We also had 253 customers who we predicted would not churn, out of which 14.  This is less than desirable, but the model can (and should) be tuned to improve this.  Most importantly, note that with minimal effort, our model produced accuracies similar to those published [here](http://media.salford-systems.com/video/tutorial/2015/targeted_marketing.pdf).

_Note that because there is some element of randomness in the algorithm's subsample, your results may differ slightly from the text written above._

Hyperparameter optimization and balancing of the classes is typically recommended to improve model performance. This is not part of today's session. 

---

## Extensions

This example analyzed a relatively small dataset, but utilized Amazon SageMaker features such as distributed, managed training and real-time model hosting, which could easily be applied to much larger problems.  In order to improve predictive accuracy further, we could tweak value we threshold our predictions at to alter the mix of false-positives and false-negatives, or we could explore techniques like hyperparameter tuning.  In a real-world scenario, we would also spend more time engineering features by hand and would likely look for additional datasets to include which contain customer information not available in our initial dataset.

## Clean-up (Optional)

If you are done with this notebook, please run the cell below.  This will remove the hosted endpoint you created and avoid any charges from a stray instance being left on.

In [ ]:
# cell 10
xgb_predictor.delete_endpoint(delete_endpoint_config=True)